# 数据预处理

In [2]:
import pandas as pd
import numpy as np
final = []
for i in range(2010, 2025):
    name = "/home/cheam/IP_alpha/data/factor_" + str(i) + ".parquet"
    print(name)
    df = pd.read_parquet(name)
    final.append(df)
final= pd.concat(final)
final2 = final.set_index(['Symbol', 'Date'])

/home/cheam/IP_alpha/data/factor_2010.parquet
/home/cheam/IP_alpha/data/factor_2011.parquet
/home/cheam/IP_alpha/data/factor_2012.parquet
/home/cheam/IP_alpha/data/factor_2013.parquet
/home/cheam/IP_alpha/data/factor_2014.parquet
/home/cheam/IP_alpha/data/factor_2015.parquet
/home/cheam/IP_alpha/data/factor_2016.parquet
/home/cheam/IP_alpha/data/factor_2017.parquet
/home/cheam/IP_alpha/data/factor_2018.parquet
/home/cheam/IP_alpha/data/factor_2019.parquet
/home/cheam/IP_alpha/data/factor_2020.parquet
/home/cheam/IP_alpha/data/factor_2021.parquet
/home/cheam/IP_alpha/data/factor_2022.parquet
/home/cheam/IP_alpha/data/factor_2023.parquet
/home/cheam/IP_alpha/data/factor_2024.parquet


In [5]:
from typing import List
def resample_to_week(df):
    """周频重采样函数"""
    weekly_data = df.groupby('Symbol').resample('W-FRI', level='Date').last()  # 每周的第一个交易日的因子均值
    # 删除包含 NaN 的行
    weekly_data = weekly_data.dropna()
    return weekly_data

def standardize_by_date_total(df: pd.DataFrame, columns_to_scale:List[str])-> pd.DataFrame:
    """
    根据天分组对数据进行截面z-score标准化处理。

    参数:
    - df: 未处理的数据集。
    - columns_to_scale: 要处理的列。

    返回:
    - pd.DataFrame: 标准化后的数据。
    """
    standardized_df = df.copy()
    for date, group in df.groupby('Date'):
        scaler = StandardScaler()
        standardized_values = scaler.fit_transform(group[columns_to_scale])
        standardized_df.loc[group.index, columns_to_scale] = standardized_values
    return standardized_df

def cal_label(df: pd.DataFrame, next: int, col_name = 'AdjVWAP')-> pd.DataFrame:
    """
    计算标签，并将结果替换至回当天的数据中。

    参数:
    - df: 未处理数据。
    - next: 预测的天数。eg: next = 1，则表示预测明天收益率。
    - col_name: 'AdjVWAP'计算的列名。

    返回:
    - pd.DataFrame: 计算未来收益率后的数据。
    """
    label = df[[col_name]].unstack(level='Symbol').pct_change(next).shift(-next-1).stack(level='Symbol')
    df[col_name] = label
    df = df[~df.index.get_level_values(0).isin(df.index.get_level_values(0).unique()[-next-1:])]
    return df

In [6]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import joblib
from sklearn.linear_model import HuberRegressor
from sklearn.linear_model import ElasticNet
from sklearn.datasets import make_regression
import numpy as np
import optuna
import operator
from sklearn.model_selection import train_test_split
#from xgboost import XGBRegressor

from pandas import read_parquet
from sklearn.model_selection import KFold
from sklearn.metrics import root_mean_squared_error
import xgboost as xgb
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
import lightgbm as lgb

import pickle
import matplotlib.pyplot as plt
final2= final2.reset_index()
final2['Date'] = pd.to_datetime(final2['Date'])
final2 = final2.set_index(['Date', 'Symbol'])
final2.sort_values(by=['Date', 'Symbol'], inplace=True)
final2 = standardize_by_date_total(final2, final2.columns.drop('AdjVWAP'))
final = resample_to_week(final2)
df = final.copy()
next = 1
col_name = 'AdjVWAP'
df = df.reset_index()
df = df.set_index(['Date','Symbol'])
df.sort_index(level=['Date', 'Symbol'], inplace=True)
df[col_name] = df[['AdjVWAP']].unstack(level='Symbol').pct_change(next).shift(-next-1).stack(level='Symbol')
df = df[~df.index.get_level_values('Date').isin(df.index.get_level_values('Date').unique()[-next-1:])]

# 生成日度的预测值

In [ ]:
import pandas as pd
import numpy as np
data = pd.read_parquet('/home/cheam/IP_alpha/data/factor_onlystd.parquet')
#results_all_tunning = pd.DataFrame(columns=['test_start_date','eta','max_depth','colsample_bytree','colsample_bylevel','subsample','out_ic','in_ic','out_mse','in_mse'])
date_range = pd.date_range(start='2010-01-01', end='2025-02-01', freq='6M')# end应该是24-10-01
year = [
    
    (f"{date.year}-{date.month:02d}", 1 if date.month == 1 or date.month ==7 else 0) for date in date_range
]
import joblib
from sklearn.linear_model import HuberRegressor
from sklearn.linear_model import ElasticNet
from sklearn.datasets import make_regression
import numpy as np
import lightgbm as lgb
import optuna
import operator
import os
#from xgboost import XGBRegressor
import sys
import time
#f=open(os.path.join(data_dir, "train.txt"),"w+")
from pandas import read_parquet
from sklearn.model_selection import KFold
from sklearn.metrics import root_mean_squared_error
import xgboost as xgb

import pickle
import matplotlib.pyplot as plt
data

In [19]:
results=pd.DataFrame()
data_dir = "/home/cheam/IP_alpha/xgb_rolling_test/"
for i in range(len(year)-5): 
    train_start_date = year[0][0]
    #valid_start_date = year[i+3][0]
    test_start_date = year[i+4][0]
    valid_start_date = pd.to_datetime(test_start_date) - pd.DateOffset(months=3)
    test_end_date = year[i+5][0]
    print('train_start_date',train_start_date, 'valid_start_date',valid_start_date, 'test_start_date',test_start_date)
    train_data = data.loc[(data.index.get_level_values(0) >= train_start_date) & (data.index.get_level_values(0) < valid_start_date)]
    valid_data = data.loc[(data.index.get_level_values(0) >= valid_start_date) & (data.index.get_level_values(0) < test_start_date)]
    test_data = data.loc[(data.index.get_level_values(0) >= test_start_date) & (data.index.get_level_values(0) < test_end_date)]

    Dtrain = xgb.DMatrix(train_data.drop('AdjVWAP',axis=1), label=train_data['AdjVWAP'])
    Dvalid = xgb.DMatrix(valid_data.drop('AdjVWAP',axis=1), label=valid_data['AdjVWAP'])
    Dtest = xgb.DMatrix(test_data.drop('AdjVWAP',axis=1), label=test_data['AdjVWAP'])
    variable_part = "已调参" if  year[i][1] == 1 else ""
    model_name = data_dir + 'xgb_' + train_start_date+'_'+test_start_date+ variable_part + '.json'
    # with open(model_name, 'r') as f:
    #     model_json = f.read()

    # model = lgb.Booster(model_str=model_json)
    model = xgb.Booster()
    model.load_model(model_name)


    #y_pred = pd.DataFrame(model.predict(test_data.drop('AdjVWAP',axis=1)))
    y_pred = pd.DataFrame(model.predict(Dtest))
    y_pred.index=test_data.index
    y_pred.columns=[f"pred_"+test_start_date]
    results = pd.concat([y_pred, results], join= 'outer', axis=1)

    

train_start_date 2010-01 valid_start_date 2011-10-01 00:00:00 test_start_date 2012-01
train_start_date 2010-01 valid_start_date 2012-04-01 00:00:00 test_start_date 2012-07
train_start_date 2010-01 valid_start_date 2012-10-01 00:00:00 test_start_date 2013-01
train_start_date 2010-01 valid_start_date 2013-04-01 00:00:00 test_start_date 2013-07
train_start_date 2010-01 valid_start_date 2013-10-01 00:00:00 test_start_date 2014-01
train_start_date 2010-01 valid_start_date 2014-04-01 00:00:00 test_start_date 2014-07
train_start_date 2010-01 valid_start_date 2014-10-01 00:00:00 test_start_date 2015-01
train_start_date 2010-01 valid_start_date 2015-04-01 00:00:00 test_start_date 2015-07
train_start_date 2010-01 valid_start_date 2015-10-01 00:00:00 test_start_date 2016-01
train_start_date 2010-01 valid_start_date 2016-04-01 00:00:00 test_start_date 2016-07
train_start_date 2010-01 valid_start_date 2016-10-01 00:00:00 test_start_date 2017-01
train_start_date 2010-01 valid_start_date 2017-04-01 0

In [20]:
results['mean'] = results.mean(axis=1)
results = results.sort_index(level='Date')
results.to_parquet(data_dir + 'daily_results_test.parquet')
#results.to_parquet("/home/cheam/IP_alpha/test.parquet")
results

pred_2024-07  pred_2024-01  pred_2023-07  pred_2023-01  \
Date       Symbol                                                              
2012-01-04 000001.SZ           NaN           NaN           NaN           NaN   
           000002.SZ           NaN           NaN           NaN           NaN   
           000004.SZ           NaN           NaN           NaN           NaN   
           000005.SZ           NaN           NaN           NaN           NaN   
           000006.SZ           NaN           NaN           NaN           NaN   
...                            ...           ...           ...           ...   
2024-10-11 688799.SH      0.003543           NaN           NaN           NaN   
           688800.SH      0.001078           NaN           NaN           NaN   
           688819.SH      0.002025           NaN           NaN           NaN   
           688981.SH     -0.001219           NaN           NaN           NaN   
           689009.SH      0.001289           NaN           NaN           NaN   

                      pred_2022-07  pred_2022-01  pred_2021-07  pred_2021-01  \
Date       Symbol                                                              
2012-01-04 000001.SZ           NaN           NaN           NaN           NaN   
           000002.SZ           NaN           NaN           NaN           NaN   
           000004.SZ           NaN           NaN           NaN           NaN   
           000005.SZ           NaN           NaN           NaN           NaN   
           000006.SZ           NaN           NaN           NaN           NaN   
...                            ...           ...           ...           ...   
2024-10-11 688799.SH           NaN           NaN           NaN           NaN   
           688800.SH           NaN           NaN           NaN           NaN   
           688819.SH           NaN           NaN           NaN           NaN   
           688981.SH           NaN           NaN           NaN           NaN   
           689009.SH           NaN           NaN           NaN           NaN   

                      pred_2020-07  pred_2020-01  ...  pred_2016-01  \
Date       Symbol                                 ...                 
2012-01-04 000001.SZ           NaN           NaN  ...           NaN   
           000002.SZ           NaN           NaN  ...           NaN   
           000004.SZ           NaN           NaN  ...           NaN   
           000005.SZ           NaN           NaN  ...           NaN   
           000006.SZ           NaN           NaN  ...           NaN   
...                            ...           ...  ...           ...   
2024-10-11 688799.SH           NaN           NaN  ...           NaN   
           688800.SH           NaN           NaN  ...           NaN   
           688819.SH           NaN           NaN  ...           NaN   
           688981.SH           NaN           NaN  ...           NaN   
           689009.SH           NaN           NaN  ...           NaN   

                      pred_2015-07  pred_2015-01  pred_2014-07  pred_2014-01  \
Date       Symbol                                                              
2012-01-04 000001.SZ           NaN           NaN           NaN           NaN   
           000002.SZ           NaN           NaN           NaN           NaN   
           000004.SZ           NaN           NaN           NaN           NaN   
           000005.SZ           NaN           NaN           NaN           NaN   
           000006.SZ           NaN           NaN           NaN           NaN   
...                            ...           ...           ...           ...   
2024-10-11 688799.SH           NaN           NaN           NaN           NaN   
           688800.SH           NaN           NaN           NaN           NaN   
           688819.SH           NaN           NaN           NaN           NaN   
           688981.SH           NaN           NaN           NaN           NaN   
           689009.SH           NaN           NaN        

# 查看optimization 的年化收益

In [18]:
import pandas as pd
record_dir = "/home/cheam/IP_alpha/backtest_result_compare/xgb_diff__2012-01-01_2024-10-01_bm000905SH_tf5_bl30_s05k"
name = "XGBoost"
all_recored = pd.read_excel( record_dir + "/all_record.xlsx")
yr_record = pd.read_excel(record_dir + "/yr_record.xlsx")
all_recored['trade_year'] = "all"
all_recored.rename(columns={'return_all':'return_yr', 'sharpe_all':'sharpe_yr', 'max_drawdown_all':'max_draw_down'},inplace=True)
record_2y = pd.concat([all_recored, yr_record],axis=0)
record_2y = record_2y[['trade_year','return_yr','sharpe_yr','max_draw_down']]
record_2y.rename(columns={'return_yr':'return_'+name,'sharpe_yr':'sharpe_'+name,'max_draw_down' : 'max_drawdown_'+name},inplace=True)
record_2y

,trade_year,return_XGBoost,sharpe_XGBoost,max_drawdown_XGBoost
0,all,0.08611,0.490439,-0.363726
0,2012,0.17320,0.878068,-0.092101
1,2013,0.19327,1.100396,-0.052524
2,2014,-0.00256,0.060636,-0.099857
3,2015,-0.07868,-0.057871,-0.203280
4,2016,0.33769,1.232614,-0.136907
5,2017,-0.03998,-0.189210,-0.169460
6,2018,0.24537,1.119568,-0.135493
7,2019,0.30169,1.233179,-0.084332
8,2020,-0.21389,-0.913211,-0.291279
